In [1]:
from modules.utils import load_yaml, rle_decode, rle_encode, draw_fig
from modules.optimizer import get_optimizer
from modules.scheduler import get_scheduler
from modules.loss import get_loss_function
from modules.model import get_transformers_model, get_smp_model
from modules.dataset import transformsCustomDataset, smpDataset
from modules.augmentation import *

import os
import random
import shutil
import argparse
from glob import glob
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

/home/vip/anaconda3/envs/seg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prj_dir = './'
config_path = os.path.join(prj_dir, 'config', 'predict_smp_512.yaml')
config = load_yaml(config_path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
df = pd.read_csv(f"data/test.csv")

images = []
for img in df["img_path"]:
    path = f"./data/test_img/{os.path.basename(img)}"
    images.append(path)

print('test len:', len(images))

test len: 60640


In [4]:
test_transfrom = A.Compose([
    A.Equalize(),
    A.Normalize(),
    ToTensorV2(),
])

test_dataset = smpDataset(images=images, 
                          masks=None, 
                          transform=test_transfrom, 
                          infer=True)

test_dataloader = DataLoader(test_dataset, 
                              batch_size=config['batch_size'], 
                              num_workers=config['num_workers'],
                              shuffle=False)

In [5]:
model = get_smp_model(name=config['model']['architecture'])

model = model(encoder_name=config['model']['encoder'],
              encoder_weights=config['model']['encoder_weight'],
              in_channels=config['model']['in_channel'],
              classes=config['model']['n_classes'],
)
model.to(device)

weights = torch.load(config['model']['pretrained'])
model.load_state_dict(weights)

<All keys matched successfully>

In [6]:
result = []

with torch.no_grad():
    model.eval()
    for idx, (images, filenames) in enumerate(tqdm(test_dataloader)):
        images = images.to(device)
        outputs = model(images)
        
        seg_prob = torch.sigmoid(outputs).detach().cpu().numpy().squeeze()
        seg = (seg_prob > 0.5).astype(np.uint8)
        
        for i in range(len(images)):
            mask_rle = rle_encode(seg[i])
            if mask_rle == '':
                result.append(-1)
            else:
                result.append(mask_rle)

100%|██████████| 1895/1895 [04:40<00:00,  6.76it/s]


# Submission

In [7]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['mask_rle'] = result

In [8]:
submit.to_csv('./submit3.csv', index=False)